# NoteBook To Check Point or Polygon Inside Polygon

The objective of this notebook is to search a set of points or polygons for spatial overlap with a largerset

By Cascsde Tuholske 2019-01-17

#### Notes
1. Points should be dialated to circles to prevent one falling just outside the larger polygon set (see Lusaka)

### Depedencies

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import mapping
from shapely.geometry import Polygon
from shapely.geometry import shape
import ast
from shapely.geometry import mapping
import rasterio
import copy

### Functions

In [2]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [3]:
def point_buffer(gpd_df, raduis):
    "Function to make a shapely polygon buffer around a point"
   
    # AGU 2018-12-04 - radius set to ~250m at the equator 
    #radius = 250*1/(111*1000)

    # new_gpd_df = gpd.GeoDataFrame()
    gpd_df.rename(columns={'geometry':'old_geom'}, inplace=True)
    
    arr = []
    
    for point in gpd_df['old_geom']:
        buffer = point.buffer(radius)
        arr.append((buffer))
    
    gpd_df['geometry'] = arr
    
    return gpd_df

In [4]:
def poly_point (poly, point):
    """
    This function will check if points are inside polygons if given two gpd dataframes with points and polygons
    Returns point ids, point geometry, polygon index # and polygon geometry
    """
    
    out_arr = [] #return an array <<< ---------------- ASK RYAN IF BETTER DO USE DICT 
    
    for index_point, row_point in point.iterrows():
        for index_poly, row_poly in poly.iterrows():
            if row_point['geometry'].within(row_poly['geometry']):
                point_id = row_point['id']
                point_geom = mapping(row_point['geometry']) # makes a dict w/ keys : type and cood
                poly_id = index_poly
                poly_geom = mapping(row_poly['geometry']) # makes a dict w/ keys : type and cood
                
                out_arr.append((point_id, 
                                point_geom, 
                                poly_id, 
                                poly_geom))

    return out_arr

# Note 2018-11-30 update arr to gpd_df 

In [5]:
def poly_overlap (point_buffer, poly_raster):
    """
    This function will check if point buffers intersect with polygons 
    if given two gpd dataframes with point buffers and polygons
    Returns point ids, point geometry, polygon index #, and polygon geometry in a geopandas DF.
    It goes faster if smaller list goes first
    """
    
    # make arrays to fill
    country_arr = []
    city_arr = []
    osm_type_arr = []
    lat_arr = []
    lon_arr = []
    osm_id_arr = [] 
    FID_arr = [] 
    poly_geom_arr = []
    #point_geom_arr = []
    
    for index_point_buffer, row_point_buffer in point_buffer.iterrows():
        for index_poly_raster, row_poly_raster in poly_raster.iterrows():
            if row_point_buffer['geometry'].intersects(row_poly_raster['geometry']):
                country = row_point_buffer['country']
                city = row_point_buffer['city']
                osm_type = row_point_buffer['type']
                lat = row_point_buffer['lat']
                lon = row_point_buffer['lon']
                osm_id = row_point_buffer['osm_id']
                poly_id = row_poly_raster['FID']
                poly_geom = shape(mapping(row_poly_raster['geometry'])) # make polygon
                #point_geom = row_point_buffer['old_geom']

                country_arr.append((country))
                city_arr.append((city))
                osm_type_arr.append((osm_type))
                lat_arr.append((lat))
                lon_arr.append((lon))
                osm_id_arr.append((osm_id))
                FID_arr.append((poly_id))
                poly_geom_arr.append((poly_geom))
                #point_geom_arr.append((point_geom))
    
    # put results into a geopandas df
    new_gpd_df = gpd.GeoDataFrame()
    new_gpd_df['osm_id'] = osm_id_arr
    new_gpd_df['FID'] = FID_arr
    new_gpd_df['geometry'] = poly_geom_arr
    new_gpd_df['country'] = country_arr           
    new_gpd_df['city'] = city_arr
    new_gpd_df['osm_type'] = osm_type
    new_gpd_df['lat'] = lat_arr
    new_gpd_df['lon'] = lon_arr
    #new_gpd_df['point_geom'] = point_geom_arr
    
    
    return new_gpd_df

# Note 2018-11-30 update arr to gpd_df 

### Analysis

In [6]:
# File Paths

# data = ''
temp_data = '../../temp_data/'
osm_point_file = '20190114_osm_africa_cities.csv'

In [7]:
# Load OSM Points
osm_point_gdf = load_points(temp_data+osm_point_file)
print(len(osm_point_gdf))

876


In [8]:
osm_point_gdf.head(6)

,country,osm_id,city,lat,lon,geometry
0,Algeria,27564946,Constantine,36.364519,6.608260,POINT (6.60826 36.364519)
1,Algeria,27565103,Oran,35.703275,-0.649298,POINT (-0.6492975999999999 35.7032751)
2,Algeria,253292160,Tizi Ouzou,36.713784,4.049392,POINT (4.0493919 36.7137843)
3,Algeria,262963217,El Oued,33.361177,6.860349,POINT (6.8603492 33.3611766)
4,Algeria,262964638,Touggourt,33.109897,6.066102,POINT (6.066102 33.1098968)
5,Algeria,264573224,Laghouat,33.806352,2.880862,POINT (2.8808616 33.8063518)


In [9]:
# Add column for type: city or town
osm_point_gdf['type'] = 'city'
osm_point_gdf.head(6)

,country,osm_id,city,lat,lon,geometry,type
0,Algeria,27564946,Constantine,36.364519,6.608260,POINT (6.60826 36.364519),city
1,Algeria,27565103,Oran,35.703275,-0.649298,POINT (-0.6492975999999999 35.7032751),city
2,Algeria,253292160,Tizi Ouzou,36.713784,4.049392,POINT (4.0493919 36.7137843),city
3,Algeria,262963217,El Oued,33.361177,6.860349,POINT (6.8603492 33.3611766),city
4,Algeria,262964638,Touggourt,33.109897,6.066102,POINT (6.066102 33.1098968),city
5,Algeria,264573224,Laghouat,33.806352,2.880862,POINT (2.8808616 33.8063518),city


In [10]:
# AGU 2018-12-04 - radius set to ~250m at the equator 

radius = 250*1/(111*1000)
radius

0.0022522522522522522

In [11]:
osm_buffer_gdf = point_buffer(osm_point_gdf, radius)

In [12]:
print(len(osm_buffer_gdf))

876


In [13]:
osm_buffer_gdf.head(6)

,country,osm_id,city,lat,lon,old_geom,type,geometry
0,Algeria,27564946,Constantine,36.364519,6.608260,POINT (6.60826 36.364519),city,"POLYGON ((6.610512252252253 36.364519, 6.61050..."
1,Algeria,27565103,Oran,35.703275,-0.649298,POINT (-0.6492975999999999 35.7032751),city,"POLYGON ((-0.6470453477477477 35.7032751, -0.6..."
2,Algeria,253292160,Tizi Ouzou,36.713784,4.049392,POINT (4.0493919 36.7137843),city,"POLYGON ((4.051644152252252 36.7137843, 4.0516..."
3,Algeria,262963217,El Oued,33.361177,6.860349,POINT (6.8603492 33.3611766),city,"POLYGON ((6.862601452252252 33.3611766, 6.8625..."
4,Algeria,262964638,Touggourt,33.109897,6.066102,POINT (6.066102 33.1098968),city,"POLYGON ((6.068354252252252 33.1098968, 6.0683..."
5,Algeria,264573224,Laghouat,33.806352,2.880862,POINT (2.8808616 33.8063518),city,"POLYGON ((2.883113852252253 33.8063518, 2.8831..."


In [14]:
osm_buffer_gdf = osm_buffer_gdf.drop(['old_geom'], axis=1)
osm_buffer_gdf.head(6)

,country,osm_id,city,lat,lon,type,geometry
0,Algeria,27564946,Constantine,36.364519,6.608260,city,"POLYGON ((6.610512252252253 36.364519, 6.61050..."
1,Algeria,27565103,Oran,35.703275,-0.649298,city,"POLYGON ((-0.6470453477477477 35.7032751, -0.6..."
2,Algeria,253292160,Tizi Ouzou,36.713784,4.049392,city,"POLYGON ((4.051644152252252 36.7137843, 4.0516..."
3,Algeria,262963217,El Oued,33.361177,6.860349,city,"POLYGON ((6.862601452252252 33.3611766, 6.8625..."
4,Algeria,262964638,Touggourt,33.109897,6.066102,city,"POLYGON ((6.068354252252252 33.1098968, 6.0683..."
5,Algeria,264573224,Laghouat,33.806352,2.880862,city,"POLYGON ((2.883113852252253 33.8063518, 2.8831..."


In [21]:
# Load Polygon
poly_raster_file = 'GHS_POP_GPW42015_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp'
raster_polygons = gpd.read_file(temp_data+poly_raster_file)
print(len(raster_polygons))

46074


In [22]:
raster_polygons.head(6)

,FID,geometry
0,0,"POLYGON ((1776405.452556 -4197970.538132, 1778..."
1,1,"POLYGON ((856405.452556 4490029.461868, 857405..."
2,2,"POLYGON ((1767405.452556 -4161970.538132, 1769..."
3,3,"POLYGON ((1717405.452556 -4175970.538132, 1718..."
4,4,"POLYGON ((1729405.452556 -4158970.538132, 1730..."
5,5,"POLYGON ((1703405.452556 -4154970.538132, 1708..."


In [24]:
raster_polygons.crs = {'ellps': 'WGS84',
 'lon_0': 0,
 'no_defs': True,
 'proj': 'moll',
 'units': 'm',
 'x_0': 0,
 'y_0': 0}

In [ ]:
# {'ellps': 'WGS84',
#  'lon_0': 0,
#  'no_defs': True,
#  'proj': 'moll',
#  'units': 'm',
#  'x_0': 0,
#  'y_0': 0}

In [ ]:
# For GHS make FID and reset crs
# FID = list(range(len(raster_polygons)))
# raster_polygons['DN'] = FID
# raster_polygons.columns.values[0] = "FID"

In [25]:
raster_polygons_crs = raster_polygons.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [27]:
raster_polygons_crs.head(6)

,FID,geometry
0,0,POLYGON ((20.02554510861832 -34.77980766795523...
1,1,"POLYGON ((9.852422643144987 37.34058756990897,..."
2,2,POLYGON ((19.87721487508657 -34.46627715636649...
3,3,"POLYGON ((19.33251509633847 -34.5881527049174,..."
4,4,POLYGON ((19.44605623645006 -34.44016969569522...
5,5,POLYGON ((19.14873348283236 -34.40536452690367...


In [ ]:
# test

# osm_sub = osm_buffer_gdf.iloc[0:100]
# osm_sub.head(6)

In [ ]:
# poly_sub = raster_polygons.iloc[0:1000]
# poly_sub.head(6)

In [ ]:
# subset test

# import time
# checkpoint = time.time()

# test = poly_overlap(osm_sub, poly_sub)

# print("elapsed time is: {}s".format(time.time()-checkpoint))

In [ ]:
# len(test)

In [ ]:
# test

In [28]:
# run poly_overlap

import time
checkpoint = time.time()

poly_overlap_out = poly_overlap(osm_buffer_gdf, raster_polygons_crs)

print("elapsed time is: {}s".format(time.time()-checkpoint))

elapsed time is: 2639.496916294098s


In [29]:
print(len(poly_overlap_out))

777


In [ ]:
poly_overlap_out.head(6)

In [30]:
poly_overlap_out.to_file(temp_data+'GHS_POP_GPW42015_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_polyoverlap.shp', driver='ESRI Shapefile')